In [ ]:
import pandas as pd, re, unicodedata, difflib

def normalize(text):
    if pd.isna(text): return ''
    s = unicodedata.normalize('NFKD', str(text)).lower()
    s = re.sub(r'\b(ооо|пао|ао|зао|pjsc|ojsc|ltd|gmbh|гк)\b', ' ', s)
    s = re.sub(r'[«»"“”\'‘’]', '', s)
    s = re.sub(r'[-–]', ' ', s)
    s = re.sub(r'[^\w\s]', '', s)
    return re.sub(r'\s+', ' ', s).strip()

reviews = pd.read_csv('reviews_with_sentiment.csv')
info    = pd.read_csv('cian_zastroischiki_info_all.csv')

reviews['dev_norm'] = reviews['developer'].apply(normalize)
info['dev_norm']    = info['developer_name'].apply(normalize)

manual_map_raw = {
    'гк главстрой': 'Главстрой Санкт-Петербург',
    'главстрои':   'Главстрой Санкт-Петербург',
    'группа пик':  'пик',
    'пик группа':  'пик',
    'группа фск':  'фск'
}

manual_map = {normalize(k): normalize(v) for k, v in manual_map_raw.items()}

reviews['dev_norm'] = reviews['dev_norm'].replace(manual_map)

# добавляем строки для 7 застройщиков вручную так как у них катсомные страницы на циан/нет необходимой информации
pik_row = {
    'dev_norm':           normalize('пик'),
    'year_of_foundation': 1994,
    'cnt_houses_built':   1268,
    'estate_class_list':  'Эконом; Комфорт; Бизнес',
    'avg_sqm_price':      329_270
}

fsk_row = {
    'dev_norm':           normalize('фск'),
    'year_of_foundation': 2005,
    'cnt_houses_built':   202,
    'estate_class_list':  'Комфорт; Бизнес',
    'avg_sqm_price':      341_363
}

polis_row = {
    'dev_norm':           normalize('полис'),
    'year_of_foundation': 2010,
    'cnt_houses_built':   68,
    'estate_class_list':  'Комфорт',
    'avg_sqm_price':      150_500
}

october_row = {
    'dev_norm':           normalize('october group'),
    'year_of_foundation': 2022,
    'cnt_houses_built':   0,
    'estate_class_list':  'Бизнес',
    'avg_sqm_price':      637_960
}

vozrozhdenie_row = {
    'dev_norm':           normalize('возрождение северо запад иск'),
    'year_of_foundation': 2000,
    'cnt_houses_built':   1,
    'estate_class_list':  'Комфорт',
    'avg_sqm_price':      190_000
}

interrost_row = {
    'dev_norm':           normalize('интеррост'),
    'year_of_foundation': 2014,
    'cnt_houses_built':   2,
    'estate_class_list':  'Комфорт',
    'avg_sqm_price':      278_818
}

optima_row = {
    'dev_norm':           normalize('optima development'),
    'year_of_foundation': 2007,
    'cnt_houses_built':   5,
    'estate_class_list':  'Премиум',
    'avg_sqm_price':      632_677
}

forma_row = {
    'dev_norm':           normalize('forma'),
    'year_of_foundation': 2021,
    'cnt_houses_built':   5,
    'estate_class_list':  'Премимум',
    'avg_sqm_price':      1_033_485
}

info = pd.concat([info, pd.DataFrame([pik_row, fsk_row, polis_row, october_row, vozrozhdenie_row, interrost_row, optima_row, forma_row])],
                 ignore_index=True)


cols_keep = ['dev_norm', 'year_of_foundation', 'cnt_houses_built',
             'estate_class_list', 'avg_sqm_price']

info_one = (info[cols_keep]
            .sort_values('year_of_foundation', ascending=False, na_position='last')
            .groupby('dev_norm', as_index=False)
            .agg('first'))


joined = (reviews
          .merge(info_one, on='dev_norm', how='left'))

# Проверяем
print('Строк до/после:', len(reviews), len(joined))
print('Доля без цены м²:', joined['avg_sqm_price'].isna().mean().round(3))


Строк до/после: 18048 18048
Доля без цены м²: 0.0


In [ ]:
joined.head(5)

,developer,username,review_date,rating,project_name,full_review_text,doc_id,sentiment_label,dev_norm,year_of_foundation,cnt_houses_built,estate_class_list,avg_sqm_price
0,Sminex,Василий,2024-12-13,5.0,Жилой квартал «Новое Пушкино»,"Чисто , поддерживается чистота это важно. Новы...",0,2,sminex,2001.0,44.0,Премиум,671443.0
1,Sminex,Ирина,2024-12-13,4.0,Жилой квартал «Новое Пушкино»,Нравится. Мне нравится территориальное располо...,1,2,sminex,2001.0,44.0,Премиум,671443.0
2,Sminex,Alina,2024-11-22,5.0,ЖК «Преображение»,Для меня все хорошо!. Живу тут со сдачи дома\n...,2,2,sminex,2001.0,44.0,Премиум,671443.0
3,Sminex,Галина,2024-11-17,4.0,Жилой квартал «Новое Пушкино»,"Вобщем, неплохо. Привлекла экология, удобное т...",3,2,sminex,2001.0,44.0,Премиум,671443.0
4,Sminex,Дарья Андреева,2024-11-15,4.0,ЖК «Вестердам»,Бывает и хуже. Мы выбрали исходя на тот момент...,4,2,sminex,2001.0,44.0,Премиум,671443.0


In [ ]:
df = joined

In [ ]:
"""
One-hot-encoding для столбца `estate_class_list`
• итоговые колонки строго:  'Эконом', 'Комфорт', 'Бизнес', 'Премиум'
• если класс не указан → 0
• результат →  joined_with_onehot.csv
"""

# ---------- 2. канонические названия ------------------
canon = {
    "эконом"  : "Эконом",
    "комфорт" : "Комфорт",
    "бизнес"  : "Бизнес",
    "премиум" : "Премиум",          # сюда же можно отнести "элитный"
    "элит"    : "Премиум"           # на случай «Элит» / «Элитный»
}

# заготовим все 4 столбца нулями
for col in canon.values():
    df[col] = 0

# ---------- 3. функция разбора ------------------------
def token_set(text: str) -> set[str]:
    """
    возвращает множество "сырых" токенов в нижнем регистре
      эконом-класс, Комфорт / бизнес → {'эконом', 'комфорт', 'бизнес'}
    """
    if pd.isna(text):
        return set()
    txt = unicodedata.normalize("NFKD", str(text)).lower()
    txt = re.sub(r"[\\/]", ";", txt)        # "/" "\" → ;
    txt = re.sub(r"[,\s]+", ";", txt)       # запятые и лишние пробелы → ;
    parts = [p.strip() for p in txt.split(";") if p.strip()]
    tokens = set()
    for p in parts:
        # обрежем возможное "-класс", "-класс", " класс"
        p = re.sub(r"\s*-?-?класс$", "", p)
        tokens.add(p)
    return tokens

# ---------- 4. заполняем one-hot -----------------------
for idx, raw in enumerate(df["estate_class_list"]):
    toks = token_set(raw)
    # делаем 1 там, где токен распознан → колонки canon[token]
    for t in toks:
        if t in canon:
            df.at[idx, canon[t]] = 1

#добавляем поле с кол-вом лет на рынке
CURRENT_YEAR = 2025
df["years_on_market"] = CURRENT_YEAR - df["year_of_foundation"]
df.to_csv("joined_with_onehot_to_use_final.csv", index=False)

# Clusterization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

DATA_PATH   = "joined_with_onehot_to_use.csv"
TEXT_COLUMN = "full_review_text"

df = pd.read_csv(DATA_PATH)
docs = df[TEXT_COLUMN].fillna("").tolist()

# ─── A. BERT-вектор  ───────────────────────────────────
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("distiluse-base-multilingual-cased-v2")
text_emb = model.encode(docs, show_progress_bar=True, batch_size=64)

# ─── B. Собираем доп-фичи ────────────────────────────────────────
num_cols  = ["rating", "years_on_market", "avg_sqm_price", "cnt_houses_built"]
possible = {'Эконом', 'Комфорт', 'Бизнес', 'Премиум'}
cat_cols = [c for c in df.columns if c in possible]

#  (1) числовые: лог-преобразуем «цены»/«кол-во домов» → сглаживаем
for col in ["avg_sqm_price", "cnt_houses_built"]:
    if col in df.columns:
        df[col] = np.log10(df[col].replace(0, np.nan)).fillna(0)

scaler = MinMaxScaler()
X_num  = scaler.fit_transform(df[num_cols])

#  (2) категориальные one-hot
X_cat = df[cat_cols].values

#  (3) задаём веса
X_num  *= 0.3           # рейтинг, возраст, цена
X_cat  = X_cat * 0.1    # классы жилья

# ─── C. Соединяем всё с текстом ──────────────────────────────────
X_all = np.hstack([text_emb, X_num, X_cat])

# ─── D. UMAP + HDBSCAN (как в статье) ────────────────────────────
import umap, hdbscan

umap_5d = umap.UMAP(n_neighbors=15, n_components=5,
                    metric="cosine", random_state=123).fit_transform(X_all)

clusterer = hdbscan.HDBSCAN(min_cluster_size=15,
                            metric="euclidean",
                            cluster_selection_method="eom")
labels = clusterer.fit_predict(umap_5d)


Batches:   0%|          | 0/282 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
num_cols = ['avg_sqm_price', 'years_on_market', 'cnt_houses_built']   # ваш список
print(df[num_cols].isna().sum())      # сколько NaN в каждом столбце

avg_sqm_price       0
years_on_market     0
cnt_houses_built    0
dtype: int64


In [ ]:
import pandas as pd

df = pd.read_csv("joined_with_onehot_to_use.csv")   # ваш готовый фрейм

# 1. какие строки с дыркой?
mask = df["years_on_market"].isna()

# 2. выводим уникальные девелоперы + сопутствующие данные
cols_show = ["developer", "dev_norm", "year_of_foundation",
             "avg_sqm_price", "cnt_houses_built"]   # можно добавить любые

missing_devs = (df.loc[mask, cols_show]
                  .drop_duplicates()
                  .sort_values("dev_norm"))

print(f"Девелоперов без years_on_market: {len(missing_devs)}")
print(missing_devs.to_string(index=False))


Девелоперов без years_on_market: 0
Empty DataFrame
Columns: [developer, dev_norm, year_of_foundation, avg_sqm_price, cnt_houses_built]
Index: []
